In [ ]:
from __future__ import division, print_function

# Third-party
from astropy.utils.console import ProgressBar
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as pl
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.stats import norm
import scipy.optimize as so
import emcee

# Custom
import gary.coordinates as gc
import gary.dynamics as gd
from gary.dynamics import mockstream
from gary.dynamics import orbitfit
import gary.integrate as gi
import gary.potential as gp
from gary.units import galactic

In [ ]:
# Galactocentric reference frame to use for this project
galactocentric_frame = coord.Galactocentric(z_sun=0.*u.pc,
                                            galcen_distance=8.3*u.kpc)
vcirc = 238.*u.km/u.s
vlsr = [-11.1, 12.24, 7.25]*u.km/u.s

galcen_frame = dict()
galcen_frame['galactocentric_frame'] = galactocentric_frame
galcen_frame['vcirc'] = vcirc
galcen_frame['vlsr'] = vlsr

In [ ]:
true_potential = gp.HernquistPotential(m=5E11, c=20., units=galactic)

In [ ]:
w0 = gd.CartesianPhaseSpacePosition(pos=[0,25.,0]*u.kpc,
                                    vel=[0,0,100]*u.km/u.s)
prog_orbit = true_potential.integrate_orbit(w0, dt=1., nsteps=5500, 
                                            Integrator=gi.DOPRI853Integrator)
fig = prog_orbit.plot()

In [ ]:
stream = mockstream.fardal_stream(true_potential, prog_orbit=prog_orbit, 
                                  prog_mass=5E5*u.Msun, release_every=1, Integrator=gi.DOPRI853Integrator)

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(6,6))
ax.plot(stream.pos[1], stream.pos[2], ls='none', alpha=0.25)

In [ ]:
prog_c,prog_v = prog_orbit.to_frame(coord.Galactic, **galcen_frame)
stream_c,stream_v = stream.to_frame(coord.Galactic, **galcen_frame)

In [ ]:
pl.plot(prog_c.l.wrap_at(180*u.deg).degree, prog_c.b.degree, marker=None, alpha=0.25, color='b')
pl.scatter(stream_c.l.wrap_at(180*u.deg).degree, stream_c.b.degree, 
           c=stream_c.distance.value, cmap='Greys_r', s=4)
pl.colorbar()

## Select some "stars" to observe

Only select leading tail

In [ ]:
idx = np.random.permutation(np.arange(8000, stream_c.l.size, dtype=int)[::2])[:128]
obs_c = stream_c[idx]
obs_v = [v[idx] for v in stream_v]

In [ ]:
pl.scatter(obs_c.l.wrap_at(180*u.deg).degree, obs_c.b.degree, s=4, c='k')

---

In [ ]:
np.random.seed(42)
R = orbitfit.compute_stream_rotation_matrix(obs_c, align_lon='max')

In [ ]:
# rotate all data to plot
rot_rep = orbitfit.rotate_sph_coordinate(obs_c, R)

pl.figure()
pl.plot(rot_rep.lon.wrap_at(180*u.deg).degree, rot_rep.lat.degree, ls='none', marker='o', ms=5.)

### Prepare data

In [ ]:
np.random.seed(42)

n_data = len(obs_c)
data = dict()
err = dict()

# err['distance'] = 0.02*obs_c.distance
# err['mul'] = 0.1*u.mas/u.yr
# err['mub'] = 0.1*u.mas/u.yr
# err['vr'] = 10.*u.km/u.s
err['distance'] = 1E-6*obs_c.distance
err['mul'] = 1E-1*u.mas/u.yr
err['mub'] = 1E-1*u.mas/u.yr
err['vr'] = 1E-6*u.km/u.s

data['phi1'] = rot_rep.lon
data['phi2'] = rot_rep.lat
data['distance'] = obs_c.distance + np.random.normal(0., err['distance'].value, size=n_data)*obs_c.distance.unit
data['mul'] = obs_v[0] + np.random.normal(0., err['mul'].value, size=n_data)*err['mul'].unit
data['mub'] = obs_v[1] + np.random.normal(0., err['mub'].value, size=n_data)*err['mub'].unit
data['vr'] = obs_v[2] + np.random.normal(0., err['vr'].value, size=n_data)*err['vr'].unit

---

## MCMC stuff

In [ ]:
def integrate_forward_backward(potential, w0, t_forw, t_back, dt=0.5,
                               Integrator=gi.DOPRI853Integrator, t0=0.):
    """
    Integrate an orbit forward and backward from a point and combine
    into a single orbit object.

    Parameters
    ----------
    potential : :class:`gary.potential.PotentialBase`
    w0 : :class:`gary.dynamics.CartesianPhaseSpacePosition`, array_like
    t_forw : numeric
        The amount of time to integate forward in time (a positive number).
    t_back : numeric
        The amount of time to integate backwards in time (a negative number).
    dt : numeric (optional)
        The timestep.
    Integrator : :class:`gary.integrate.Integrator` (optional)
        The integrator class to use.
    t0 : numeric (optional)
        The initial time.

    Returns
    -------
    orbit : :class:`gary.dynamics.CartesianOrbit`
    """
    if t_back != 0:
        o1 = potential.integrate_orbit(w0, dt=-dt, t1=t0, t2=t_back, Integrator=Integrator)
    else:
        o1 = None
    
    if t_forw != 0:
        o2 = potential.integrate_orbit(w0, dt=dt, t1=t0, t2=t_forw, Integrator=Integrator)
    else:
        o2 = None
    
    if o1 is None:
        return o2
    elif o2 is None:
        return o1
    
    o1 = o1[::-1]
    o2 = o2[1:]
    orbit = combine((o1, o2), along_time_axis=True)

    if orbit.pos.shape[-1] == 1:
        return orbit[:,0]
    else:
        return orbit

In [ ]:
def _unpack(p, freeze=None):
    """ Unpack a parameter vector """
    
    if freeze is None:
        freeze = dict()

    # these are for the initial conditions
    phi2,d,mul,mub,vr = p[:5]
    count_ix = 5

    # time to integrate forward and backward
    if 't_forw' not in freeze:
        t_forw = p[count_ix]
        count_ix += 1
    else:
        t_forw = freeze['t_forw']

    if 't_back' not in freeze:
        t_back = p[count_ix]
        count_ix += 1
    else:
        t_back = freeze['t_back']

    # prior on instrinsic width of stream
    if 'phi2_sigma' not in freeze:
        phi2_sigma = p[count_ix]
        count_ix += 1
    else:
        phi2_sigma = freeze['phi2_sigma']

    # prior on instrinsic depth (distance) of stream
    if 'd_sigma' not in freeze:
        d_sigma = p[count_ix]
        count_ix += 1
    else:
        d_sigma = freeze['d_sigma']

    # prior on instrinsic LOS velocity dispersion of stream
    if 'vr_sigma' not in freeze:
        vr_sigma = p[count_ix]
        count_ix += 1
    else:
        vr_sigma = freeze['vr_sigma']
        
    if 'hernquist_logm' not in freeze:
        hernquist_logm = p[count_ix]
        count_ix += 1
    else:
        hernquist_logm = freeze['hernquist_logm']

    return phi2,d,mul,mub,vr,t_forw,t_back,phi2_sigma,d_sigma,vr_sigma,hernquist_logm

In [ ]:
def ln_prior(p, data, err, R, Potential, dt, freeze=None):
    """
    Evaluate the prior over stream orbit fit parameters.

    See docstring for `ln_likelihood()` for information on args and kwargs.

    """

    # log prior value
    lp = 0.

    # unpack the parameters and the frozen parameters
    phi2,d,mul,mub,vr,t_forw,t_back,phi2_sigma,d_sigma,vr_sigma,hernquist_logm = _unpack(p, freeze)

    # time to integrate forward and backward
    t_integ = np.abs(t_forw) + np.abs(t_back)
    if t_forw <= t_back:
        raise ValueError("Forward integration time less than or equal to "
                         "backwards integration time.")

    if (t_forw != 0 and t_forw < dt) or (t_back != 0 and t_back > -dt):
        return -np.inf

    # prior on instrinsic width of stream
    if 'phi2_sigma' not in freeze:
        if phi2_sigma <= 0.:
            return -np.inf
        lp += -np.log(phi2_sigma)

    # prior on instrinsic depth (distance) of stream
    if 'd_sigma' not in freeze:
        if d_sigma <= 0.:
            return -np.inf
        lp += -np.log(d_sigma)

    # prior on instrinsic LOS velocity dispersion of stream
    if 'vr_sigma' not in freeze:
        if vr_sigma <= 0.:
            return -np.inf
        lp += -np.log(vr_sigma)

    # strong prior on phi2
    if phi2 < -np.pi/2. or phi2 > np.pi/2:
        return -np.inf
    lp += norm.logpdf(phi2, loc=0., scale=phi2_sigma)

    # uniform prior on integration time
    ntimes = int(t_integ / dt) + 1
    if t_integ <= 2. or t_integ > 1000. or ntimes < 4:
        return -np.inf
    
    if hernquist_logm < 10.5 or hernquist_logm > 12.5:
        return -np.inf

    return lp

`R` goes from Galactic to stream coords

In [ ]:
from gary.util import atleast_2d

def _mcmc_sample_to_coord(p, R):
    p = atleast_2d(p, insert_axis=-1) # note: from Gary, not Numpy
    rep = coord.SphericalRepresentation(lon=p[0]*0.*u.radian,
                                        lat=p[0]*u.radian, # this index looks weird but is right
                                        distance=p[1]*u.kpc)
    return coord.Galactic(orbitfit.rotate_sph_coordinate(rep, R.T))

def _mcmc_sample_to_w0(p, R):
    p = atleast_2d(p, insert_axis=-1) # note: from Gary, not Numpy
    c = _mcmc_sample_to_coord(p, R)
    x0 = c.transform_to(galactocentric_frame).cartesian.xyz.decompose(galactic).value
    v0 = gc.vhel_to_gal(c, pm=(p[2]*u.rad/u.Myr,p[3]*u.rad/u.Myr), rv=p[4]*u.kpc/u.Myr,
                        **galcen_frame).decompose(galactic).value
    w0 = np.concatenate((x0, v0))
    return w0

def ln_likelihood(p, data, err, R, Potential, dt, freeze=None):
    """ Evaluate the stream orbit fit likelihood. """
    chi2 = 0.

    # unpack the parameters and the frozen parameters
    phi2,d,mul,mub,vr,t_forw,t_back,phi2_sigma,d_sigma,vr_sigma,hernquist_logm = _unpack(p, freeze)

    w0 = _mcmc_sample_to_w0([phi2,d,mul,mub,vr], R)[:,0]

    # HACK: a prior on velocities
    vmag2 = np.sum(w0[3:]**2)
    chi2 += -vmag2 / (0.15**2)

    # integrate the orbit
    potential = Potential(m=10**hernquist_logm, c=true_potential.parameters['c'], units=galactic)
    orbit = integrate_forward_backward(potential, w0, t_back=t_back, t_forw=t_forw)

    # rotate the model points to stream coordinates
    model_c,model_v = orbit.to_frame(coord.Galactic, **galcen_frame)
    model_oph = orbitfit.rotate_sph_coordinate(model_c.spherical, R)
#      = model_c.transform_to(Ophiuchus)

    # model stream points in ophiuchus coordinates
    model_phi1 = model_oph.lon
    model_phi2 = model_oph.lat.radian
    model_d = model_oph.distance.decompose(galactic).value
    model_mul,model_mub,model_vr = [x.decompose(galactic).value for x in model_v]

    # for independent variable, use cos(phi)
    data_x = np.cos(data['phi1'])
    model_x = np.cos(model_phi1)
    # data_x = ophdata.coord_oph.phi1.wrap_at(180*u.deg).radian
    # model_x = model_phi1.wrap_at(180*u.deg).radian
    ix = np.argsort(model_x)

    # shortening for readability -- the data
    phi2 = data['phi2'].radian
    dist = data['distance'].decompose(galactic).value
    mul = data['mul'].decompose(galactic).value
    mub = data['mub'].decompose(galactic).value
    vr = data['vr'].decompose(galactic).value

    # define interpolating functions
    order = 3
    # bbox = [-np.pi, np.pi]
    bbox = [-1, 1]
    phi2_interp = InterpolatedUnivariateSpline(model_x[ix], model_phi2[ix], k=order, bbox=bbox) # change bbox to units of model_x
    d_interp = InterpolatedUnivariateSpline(model_x[ix], model_d[ix], k=order, bbox=bbox)
    mul_interp = InterpolatedUnivariateSpline(model_x[ix], model_mul[ix], k=order, bbox=bbox)
    mub_interp = InterpolatedUnivariateSpline(model_x[ix], model_mub[ix], k=order, bbox=bbox)
    vr_interp = InterpolatedUnivariateSpline(model_x[ix], model_vr[ix], k=order, bbox=bbox)

    chi2 += -(phi2_interp(data_x) - phi2)**2 / phi2_sigma**2 - 2*np.log(phi2_sigma)

    _err = err['distance'].decompose(galactic).value
    chi2 += -(d_interp(data_x) - dist)**2 / (_err**2 + d_sigma**2) - np.log(_err**2 + d_sigma**2)

    _err = err['mul'].decompose(galactic).value
    chi2 += -(mul_interp(data_x) - mul)**2 / (_err**2) - 2*np.log(_err)

    _err = err['mub'].decompose(galactic).value
    chi2 += -(mub_interp(data_x) - mub)**2 / (_err**2) - 2*np.log(_err)

    _err = err['vr'].decompose(galactic).value
    chi2 += -(vr_interp(data_x) - vr)**2 / (_err**2 + vr_sigma**2) - np.log(_err**2 + vr_sigma**2)

    # this is some kind of whack prior - don't integrate more than we have to
#     chi2 += -(model_phi1.radian.min() - data['phi1'].radian.min())**2 / (phi2_sigma**2)
#     chi2 += -(model_phi1.radian.max() - data['phi1'].radian.max())**2 / (phi2_sigma**2)

    return 0.5*chi2

In [ ]:
def ln_posterior(p, *args, **kwargs):
    """
    Evaluate the stream orbit fit posterior probability.

    See docstring for `ln_likelihood()` for information on args and kwargs.

    Returns
    -------
    lp : float
        The log of the posterior probability.

    """

    lp = ln_prior(p, *args, **kwargs)
    if not np.isfinite(lp):
        return -np.inf

    ll = ln_likelihood(p, *args, **kwargs)
    if not np.all(np.isfinite(ll)):
        return -np.inf

    return lp + ll.sum()

In [ ]:
def plot_mcmc_sample_orbit(p):
    w0 = _mcmc_sample_to_w0(p, R)
    orbit = true_potential.integrate_orbit(w0, dt=-0.5, t1=0., t2=freeze['t_back'])
    orbit_c,orbit_v = orbit.to_frame(coord.Galactic, **galcen_frame)

    fig,axes = pl.subplots(2,2,figsize=(12,8), sharex=True)

    # l,d
    axes[1,0].errorbar(obs_c.l.wrap_at(180*u.deg).degree, data['distance'].value, err['distance'].value, 
                       ls='none', marker='.', ecolor='#aaaaaa')
    l, = axes[1,0].plot(orbit_c.l.wrap_at(180*u.deg).degree, orbit_c.distance.value, marker=None)
    col = l.get_color()
    
    # l,b
    axes[0,0].scatter(obs_c.l.wrap_at(180*u.deg).degree, obs_c.b.degree, marker='.', c='k')
    axes[0,0].plot(orbit_c.l.wrap_at(180*u.deg).degree, orbit_c.b.degree, marker=None, c=col)

    # l,mul
    axes[0,1].errorbar(obs_c.l.wrap_at(180*u.deg).degree, data['mul'].value, err['mul'].value, 
                       ls='none', marker='.', ecolor='#aaaaaa')
    axes[0,1].plot(orbit_c.l.wrap_at(180*u.deg).degree, orbit_v[0].value, marker=None, c=col)

    # l,mub
    axes[1,1].errorbar(obs_c.l.wrap_at(180*u.deg).degree, data['mub'].value, err['mub'].value, 
                       ls='none', marker='.', ecolor='#aaaaaa')
    axes[1,1].plot(orbit_c.l.wrap_at(180*u.deg).degree, orbit_v[1].value, marker=None, c=col)
    
    axes[1,0].set_xlabel('$l$ [deg]')
    axes[1,1].set_xlabel('$l$ [deg]')
    
    fig.tight_layout()
    return fig,axes

In [ ]:
freeze = dict()
# these estimated from the plots
freeze['phi2_sigma'] = np.radians(0.9)
freeze['d_sigma'] = 0.15
freeze['vr_sigma'] = (1.5*u.km/u.s).decompose(galactic).value 
freeze['t_forw'] = 0.
freeze['t_back'] = -150.
# freeze['hernquist_logm'] = np.log10(true_potential.parameters['m'].value)

idx = data['phi1'].argmin()
p0_guess = [data['phi2'].radian[idx],
            data['distance'].decompose(galactic).value[idx],
            data['mul'].decompose(galactic).value[idx],
            data['mub'].decompose(galactic).value[idx],
            data['vr'].decompose(galactic).value[idx],
            np.log10(true_potential.parameters['m'].value)]

In [ ]:
obs_c[idx], [v[idx].decompose(galactic).value for v in obs_v], p0_guess

In [ ]:
fig,axes = plot_mcmc_sample_orbit(p0_guess)

derp = orbitfit.rotate_sph_coordinate(coord.SphericalRepresentation(lon=data['phi1'][idx][None], 
                                                                    lat=data['phi2'][idx][None],
                                                                    distance=data['distance'][idx][None]), R.T)
g = coord.Galactic(derp)
axes[0,0].plot(g.l.degree, g.b.degree, marker='o', color='r', alpha=0.4)

In [ ]:
print(ln_likelihood(p0_guess, data, err, R, gp.HernquistPotential, dt=0.5, freeze=freeze).sum(),
      ln_posterior(p0_guess, data, err, R, gp.HernquistPotential, dt=0.5, freeze=freeze))

In [ ]:
args = (data, err, R, gp.HernquistPotential, 0.5, freeze)
res = so.minimize(lambda *args,**kwargs: -ln_posterior(*args, **kwargs),
                  x0=p0_guess, method='powell', args=args)

In [ ]:
res

In [ ]:
fig,axes = plot_mcmc_sample_orbit(res.x)

In [ ]:
np.random.seed(42)
n_walkers = 8*len(p0_guess)
sampler = emcee.EnsembleSampler(nwalkers=n_walkers, dim=len(p0_guess), lnpostfn=ln_posterior,
                                args=args)

mcmc_p0 = emcee.utils.sample_ball(res.x, 1E-3*np.array(p0_guess), size=n_walkers)

In [ ]:
n_iterations = 128

with ProgressBar(n_iterations, ipython_widget=True) as bar:
    for results in sampler.sample(mcmc_p0, None, None, iterations=n_iterations):
        bar.update()
        
# _ = sampler.run_mcmc(mcmc_p0, N=128)

In [ ]:
sampler.chain[...,i].shape

In [ ]:
for i in range(5):
    pl.figure()
    for walker in sampler.chain[:,128:,i]:
        pl.plot(walker, marker=None, drawstyle='steps')